In [ ]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd


class EquityData():
    
    """       
    The EQD class is used to get a DataFrame with equity pricing data and finacial indicators 

    Attributes
    ----------
    start_date: Optional(date object)
        Date object for the start date of your DataFrame 
        
    end_date: Optional(date object)
        Date object for the end date of your DataFrame
        
    ticker_list : Optional[list[str]]
        A list of strings providing the tickers to retrieve end of day trading data
        
    indicator_list : Optional[list[str]]
        A list of strings providing the indicators to calculate for each security in the ticker_list
        
    Examples
    --------
    e = EquityData()
    print(e.df)
    
    from datetime import datetime
    start_date = datetime.now().date() - timedelta(days=30)
    ticker_list = ['UBER']
    indicator_list = ['RSI']
    e = EquityData(start_date=start_date, ticker_list=ticker_list, indicator_list=indicator_list)
    print(e.df)
    
    """
    start_date = datetime.now().date() - timedelta(days=20)
    end_date = datetime.now().date()
    
    def __init__(self, start_date=start_date, end_date=end_date, ticker_list=['AAPL', 'TSLA'], 
                 indicator_list=['WMA', 'EMA']):

        self.start_date = start_date            
        self.end_date = end_date            
        self.ticker_list = ticker_list            
        self.indicator_list = indicator_list
            
        self.df = pd.DataFrame([])
        self.get_eod_data(self)

    @staticmethod
    def get_eod_data(self):

        """
        Get EOD data for a list of tickers
        """
        eod_df = pd.DataFrame([])

        for t in self.ticker_list:
            results = pdr.DataReader(t, 'yahoo', self.start_date, self.end_date)
            results['ticker'] = t
            eod_df = eod_df.append(results)

        # Calc the list of indicators for each set of EOD data
        eod_df = eod_df.groupby(['ticker']).apply(lambda x: self.compute_analytics(x, self.indicator_list)).reset_index()

        eod_df = self.clean_data(eod_df)
        self.df = eod_df

    @staticmethod
    def compute_analytics(df, indicators):

        """
        Get calculations for each ticker in the df
        """

        for indicator in indicators:
            # Using eval so we can iterate over a list of indicators
            data = eval('TA.' + indicator + '(df)')

            # If TA returns a set -> cast set to a DataFrame
            if isinstance(data, pd.DataFrame) == False:
                data = data.to_frame()

                # To shorten the columns names I split the string 
                # and join that with the indicator name for my indicator column name
                first_string = data.columns[0].split()[0]
                data = data.rename(columns={data.columns[0]: first_string})
                data = data.add_prefix(indicator + '_')

            # Merge DataFrames based on the date
            df = df.merge(data, left_index=True, right_index=True)

        return df

    @staticmethod
    def clean_data(df):
        
        """
        Update column names to follow PEP8 and sort values
        """
        
        df = df.rename(columns={'Date': 'date', 'High': 'high', 'Low': 'low', 'Open': 'open',
                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})
        df = df.sort_values(by=['ticker', 'date'])
        return df


In [4]:
e = EquityData()
df = e.df

In [5]:
df.loc[df['ticker'] == 'AAPL']

,date,high,low,open,close,vol,adj_close,ticker,WMA_9,ema_9
0,2020-11-05,119.620003,116.870003,117.949997,119.029999,126387100.0,118.824997,AAPL,NaN,119.029999
2,2020-11-06,119.199997,116.129997,118.320000,118.690002,114457900.0,118.690002,AAPL,NaN,118.841112
4,2020-11-09,121.989998,116.050003,120.500000,116.320000,154515300.0,116.320000,AAPL,NaN,117.807869
6,2020-11-10,117.589996,114.129997,115.550003,115.970001,138023400.0,115.970001,AAPL,NaN,117.185285
8,2020-11-11,119.629997,116.440002,117.190002,119.489998,112295000.0,119.489998,AAPL,NaN,117.870885
10,2020-11-12,120.529999,118.570000,119.620003,119.209999,103162300.0,119.209999,AAPL,NaN,118.233859
12,2020-11-13,119.669998,117.870003,119.440002,119.260002,81581900.0,119.260002,AAPL,NaN,118.493549
14,2020-11-16,120.989998,118.150002,118.919998,120.300003,91183000.0,120.300003,AAPL,NaN,118.927674
16,2020-11-17,120.669998,118.959999,119.550003,119.389999,74271000.0,119.389999,AAPL,118.970889,119.034473
18,2020-11-18,119.820000,118.000000,118.610001,118.029999,76322100.0,118.029999,AAPL,118.851111,118.809413
